In [45]:
import numpy as np
import pandas as pd
import math
#hello this is khai 

In [52]:
# Read the Excel file
#basecase = pd.ExcelFile('C:\\Users\\jingy\\Documents\\EE454Lab\\system_basecase.xlsx')
# khai's file location
basecase = pd.ExcelFile("C:\\Users\\kayla\\OneDrive - UW\\EE 454\\myrepo\\EE454Project\\system_basecase.xlsx")

# Read the busdata sheet
busdata = pd.read_excel(basecase, 'BusData')
# Read the linedata sheet
linedata = pd.read_excel(basecase, 'LineData')

# Convert to numpy arrays for later use
busdata_np = np.array(busdata)
linedata_np = np.array(linedata)

# # print data to test if it reads correctly
print("Bus Data:", busdata_np.shape)
pd.DataFrame(busdata_np)




Bus Data: (12, 6)


,0,1,2,3,4,5
0,1,0.0,0.0,S,0,1.04
1,2,20.1,12.5,D,0,1.0
2,3,45.2,16.0,G,42,1.025
3,4,60.8,10.9,D,0,1.0
4,5,7.6,-1.6,D,0,1.0
5,6,13.5,8.5,D,0,1.0
6,7,33.5,11.6,D,0,1.0
7,8,9.0,5.8,D,0,1.0
8,9,4.3,2.1,G,23,1.02
9,10,5.2,1.6,G,27,1.02


In [53]:
print("Line Data:", linedata_np.shape)
pd.DataFrame(linedata_np)

Line Data: (17, 6)


,0,1,2,3,4,5
0,1.0,2.0,0.01938,0.05917,0.0528,95.0
1,1.0,5.0,0.05203,0.20304,0.0492,100.0
2,2.0,3.0,0.04699,0.19797,0.0438,9999.0
3,2.0,4.0,0.05811,0.17632,0.0340,9999.0
4,2.0,5.0,0.05695,0.07388,0.0346,9999.0
5,3.0,4.0,0.67010,0.17103,0.0128,9999.0
6,4.0,5.0,0.01335,0.04211,0.0391,9999.0
7,4.0,7.0,0.00000,0.55618,0.0000,9999.0
8,5.0,6.0,0.00000,0.25202,0.0000,9999.0
9,6.0,9.0,0.09498,0.19890,0.0176,9999.0


In [42]:
def create_Y_matrix(busdata, linedata):

    # Initialize the admittance matrix Y with all zeros
    Y = np.zeros((num_buses, num_buses), dtype=complex)

    # Process each line
    for line in linedata:
        # Get the bus number also take account the 0 index based numbering
        from_bus = int(line[0]) - 1
        to_bus = int(line[1]) - 1
        R = line[2]
        X = line[3]
        # Get the half-line shunt susceptance
        B_shunt = line[4] / 2

        Z = complex(R, X)
        # Get the line admittance
        Y_line = 1 / Z

        # Add each admittance component to the matrix
        Y[from_bus, from_bus] += Y_line + complex(0, B_shunt)
        Y[to_bus, to_bus] += Y_line + complex(0, B_shunt)
        Y[from_bus, to_bus] -= Y_line
        Y[to_bus, from_bus] -= Y_line

    # (commented out as optional) 
    # Get the conductance (G) and susceptance (B) seperately
    G = np.real(Y)
    B = np.imag(Y)

    return Y

In [54]:
# Get the admittence matrix
Y = create_Y_matrix(busdata_np, linedata_np)
G = np.real(Y)
B = np.imag(Y)
print("Admittance Matrix:\n")

pd.DataFrame(Y)


Admittance Matrix:



,0,1,2,3,4,5,6,7,8,9,10,11
0,6.183452-19.833736j,-4.999132+15.263087j,0.000000+0.000000j,0.000000+0.000000j,-1.1843210+4.6216500j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
1,-4.999132+15.263087j,14.364990-33.568623j,-1.135019+4.781863j,-1.686033+5.115838j,-6.5448060+8.4904350j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
2,0.000000+0.000000j,-1.1350190+4.7818630j,2.536066-5.111153j,-1.401047+0.357590j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
3,0.000000+0.000000j,-1.6860330+5.1158380j,-1.401047+0.357590j,9.928061-28.807011j,-6.840981+21.578554j,0.000000+0.000000j,0.000000+1.797979j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
4,-1.184321+4.621650j,-6.5448060+8.4904350j,0.000000+0.000000j,-6.840981+21.578554j,14.570107-38.597128j,0.000000+3.967939j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
5,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+3.9679390j,6.579923-17.303083j,0.000000+0.000000j,0.000000+0.000000j,-1.955029+4.094074j,-1.525967+3.175964j,-3.098927+6.102755j,0.000000+0.000000j
6,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+1.797979j,0.0000000+0.0000000j,0.000000+0.000000j,5.326055-15.155774j,-3.902050+10.365394j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,-1.424005+3.029050j
7,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,-3.902050+10.365394j,5.782934-14.734838j,-1.880885+4.402944j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
8,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+0.0000000j,-1.955029+4.094074j,0.000000+0.000000j,-1.880885+4.402944j,3.835913-8.475218j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j
9,0.000000+0.000000j,0.0000000+0.0000000j,0.000000+0.000000j,0.000000+0.000000j,0.0000000+0.0000000j,-1.525967+3.175964j,0.000000+0.000000j,0.000000+0.000000j,0.000000+0.000000j,4.014992-5.405089j,-2.489025+2.251975j,0.000000+0.000000j


In [58]:
V_mag = busdata_np[:, 5]
V_ang = np.zeros(num_buses)
busnum = busdata_np[:,0]
P_Load = busdata_np[:, 1]
Q_Load = busdata_np[:, 2]
P_Gen = busdata_np[:, 4]
bus_type = busdata_np[:, 3]

slack = None
PQind = []
PVind = []

for i in range(num_buses):


    # initialize V based on bus type
    if bus_type[i] == 'S':
        slack = i
    elif bus_type[i] == 'G':
        PVind.append(i)
    elif bus_type[i] == 'D':
        PQind.append(i)

In [15]:
print("Vmag:" + str(V_mag))
print("Vang:" + str(V_ang))

Vmag:[1.04 1.0 1.025 1.0 1.0 1.0 1.0 1.0 1.02 1.02 1.0 1.03]
Vang:[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [60]:
#formulas in textbook are written as 1 being Slack, 2 to m being PV (gen), and m+1 to N being PQ 

def Pmismatches (vpu, G, B, theta, pgen, pload):
    Pmis = np.zeros(num_buses)
    for k in range(num_buses):
        sum = 0
        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                   math.sin(math.radians(theta[k]-theta[i])))
        Pmis[k] = sum + pload[k] - pgen[k]
        
    return Pmis


def Qmismatches (vpu, G, B, theta, qload):
    Qmis = np.zeros(num_buses)
    for k in range(num_buses):
        sum = 0
        for i in range(num_buses):
            sum +=  vpu[i]*vpu[k]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                   math.cos(math.radians(theta[k]-theta[i])))
        Qmis[k] = sum + qload[k]
        
    return Qmis


    

In [61]:
print(Pmismatches(V_mag, G, B, V_ang, P_Gen, P_Load))
print(Qmismatches(V_mag, G, B, V_ang, Q_Load))

[  0.25723161  19.87165926   3.26498669  60.76497383   7.55262718
  13.43038008  33.45727984   8.9623823  -18.62174737 -21.71809416
  13.41610968 -18.02086511]
[ 0.77204344 11.68732996 16.1019658  10.84811025 -1.846316    8.31694923
 11.47247849  5.67844113  2.25065845  1.68695681  5.6425116   5.1289003 ]


In [118]:
#mostly done ? 
def HMatrix (G, B, vpu, theta):
    #HMatrix will iterate P(row) from buses 2 to N and theta(col) from buses 2 to N
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = np.arange(1, num_buses, dtype=int)
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty H array
    H = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        matchBus = False
        matchCol = None
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            #print("i is " + str(i) + "and hcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                H[r,c] = vpu[k]*vpu[i]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
                print("H has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(-1*G[k, i]*math.sin(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
            
                
        if (matchBus):     
        
            H[r,matchCol] = sum
        
        r += 1
        
    return H

In [119]:
def MMatrix (G, B, vpu, theta):
    #MMatrix will iterate P(row) from buses 2 to N and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = np.arange(1, num_buses, dtype=int)
    cols = PQind
    
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty M array
    M = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            
            #print("i is " + str(i) + "and mcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                M[r,c] = vpu[k]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
                print("M has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
                
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            M[r,matchCol] = 2*G[k, k]*vpu[k] + sum
        
        r += 1
        
    return M

In [120]:
def NMatrix (G, B, vpu, theta):
    #NMatrix will iterate Q(row) for PQ buses and theta(col) for PQ and PV buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = np.arange(1, num_buses, dtype=int)
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty N array
    N = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
#         print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
#             print("i is " + str(i) + "and Ncol is " + str(c))

            
            
            
            if (i != k and i in cols):
                N[r,c] = vpu[k]*vpu[i]*(-1*G[k, i]*math.cos(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
                print("N has been printed at " + str(r) + " " + str(c))
            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c     
            
                
            if (i != k):
                sum += vpu[k]*vpu[i]*(G[k, i]*math.cos(math.radians(theta[k]-theta[i]))+B[k, i]*
                                            math.sin(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            N[r,matchCol] = sum
        
        r += 1
        
    return N

In [121]:
def LMatrix (G, B, vpu, theta):
    #LMatrix will iterate Q(row) for PQ buses and V(col) for PQ buses
    
    #rows and cols are the index numbers of buses to be iterated
    rows = PQind
    cols = PQind
    
    #optional check for index values
#     print("rows" + str(rows))
#     print("cols" + str(cols))
    
    #empty L array
    L = np.zeros((len(rows), len(cols)))
    
    
    r = 0 
    
    for k in rows:
        sum = 0
        c = 0
        
        matchBus = False
        matchCol = None
        
        #print("k is " + str(k))
        #print("hrow is " + str(hr))
        for i in range(0, num_buses):
            
            #print("i is " + str(i) + "and lcol is " + str(c))

            
            
            
            if (i != k and i in cols):
                L[r,c] = vpu[k]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
                print("L has been printed at " + str(r) + " " + str(c))

            elif (i == k and i in cols):
                    matchBus = True
                    matchCol = c
                    
            if (i != k):
                sum += vpu[i]*(G[k, i]*math.sin(math.radians(theta[k]-theta[i]))-B[k, i]*
                                            math.cos(math.radians(theta[k]-theta[i])))
            
            if i in cols:
                c += 1
                
        if (matchBus):     
        
            L[r,matchCol] =  -2*B[k, k]*vpu[k] + sum
        
        r += 1
        
    return L

In [122]:
H = HMatrix(G, B, V_mag, V_ang)

pd.DataFrame(H)


H has been printed at 0 1
H has been printed at 0 2
H has been printed at 0 3
H has been printed at 0 4
H has been printed at 0 5
H has been printed at 0 6
H has been printed at 0 7
H has been printed at 0 8
H has been printed at 0 9
H has been printed at 0 10
H has been printed at 1 0
H has been printed at 1 2
H has been printed at 1 3
H has been printed at 1 4
H has been printed at 1 5
H has been printed at 1 6
H has been printed at 1 7
H has been printed at 1 8
H has been printed at 1 9
H has been printed at 1 10
H has been printed at 2 0
H has been printed at 2 1
H has been printed at 2 3
H has been printed at 2 4
H has been printed at 2 5
H has been printed at 2 6
H has been printed at 2 7
H has been printed at 2 8
H has been printed at 2 9
H has been printed at 2 10
H has been printed at 3 0
H has been printed at 3 1
H has been printed at 3 2
H has been printed at 3 4
H has been printed at 3 5
H has been printed at 3 6
H has been printed at 3 7
H has been printed at 3 8
H has bee

,0,1,2,3,4,5,6,7,8,9,10
0,34.381293,-4.901410,-5.115838,-8.490435,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,-4.901410,5.267939,-0.366530,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,-5.115838,-0.366530,28.858901,-21.578554,0.000000,-1.797979,0.000000,0.000000,0.000000,0.000000,0.000000
3,-8.490435,0.000000,-21.578554,38.843444,-3.967939,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,-3.967939,17.486134,0.000000,0.000000,-4.175956,-3.239483,-6.102755,0.000000
5,0.000000,0.000000,-1.797979,0.000000,0.000000,15.283295,-10.365394,0.000000,0.000000,0.000000,-3.119922
6,0.000000,0.000000,0.000000,0.000000,0.000000,-10.365394,14.856397,-4.491003,0.000000,0.000000,0.000000
7,0.000000,0.000000,0.000000,0.000000,-4.175956,0.000000,-4.491003,8.666958,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,-3.239483,0.000000,0.000000,0.000000,5.536497,-2.297014,0.000000
9,0.000000,0.000000,0.000000,0.000000,-6.102755,0.000000,0.000000,0.000000,-2.297014,10.784182,-2.384412


In [123]:
M = MMatrix(G, B, V_mag, V_ang)

pd.DataFrame(M)


M has been printed at 0 1
M has been printed at 0 2
M has been printed at 0 3
M has been printed at 0 4
M has been printed at 0 5
M has been printed at 0 6
M has been printed at 1 0
M has been printed at 1 1
M has been printed at 1 2
M has been printed at 1 3
M has been printed at 1 4
M has been printed at 1 5
M has been printed at 1 6
M has been printed at 2 0
M has been printed at 2 2
M has been printed at 2 3
M has been printed at 2 4
M has been printed at 2 5
M has been printed at 2 6
M has been printed at 3 0
M has been printed at 3 1
M has been printed at 3 3
M has been printed at 3 4
M has been printed at 3 5
M has been printed at 3 6
M has been printed at 4 0
M has been printed at 4 1
M has been printed at 4 2
M has been printed at 4 4
M has been printed at 4 5
M has been printed at 4 6
M has been printed at 5 0
M has been printed at 5 1
M has been printed at 5 2
M has been printed at 5 3
M has been printed at 5 5
M has been printed at 5 6
M has been printed at 6 0
M has been p

,0,1,2,3,4,5,6
0,14.136649,-1.686033,-6.544806,0.000000,0.000000,0.000000,0.000000
1,-1.163395,-1.436073,0.000000,0.000000,0.000000,0.000000,0.000000
2,-1.686033,9.893034,-6.840981,0.000000,0.000000,0.000000,0.000000
3,-6.544806,-6.840981,14.522734,0.000000,0.000000,0.000000,0.000000
4,0.000000,0.000000,0.000000,6.510303,0.000000,0.000000,-3.098927
5,0.000000,0.000000,0.000000,0.000000,5.283335,-3.902050,0.000000
6,0.000000,0.000000,0.000000,0.000000,-3.902050,5.745317,0.000000
7,0.000000,0.000000,0.000000,-1.994129,0.000000,-1.918502,0.000000
8,0.000000,0.000000,0.000000,-1.556487,0.000000,0.000000,-2.538805
9,0.000000,0.000000,0.000000,-3.098927,0.000000,0.000000,6.641056


In [124]:
N = NMatrix(G, B, V_mag, V_ang)

pd.DataFrame(N)


N has been printed at 0 1
N has been printed at 0 2
N has been printed at 0 3
N has been printed at 0 4
N has been printed at 0 5
N has been printed at 0 6
N has been printed at 0 7
N has been printed at 0 8
N has been printed at 0 9
N has been printed at 0 10
N has been printed at 1 0
N has been printed at 1 1
N has been printed at 1 3
N has been printed at 1 4
N has been printed at 1 5
N has been printed at 1 6
N has been printed at 1 7
N has been printed at 1 8
N has been printed at 1 9
N has been printed at 1 10
N has been printed at 2 0
N has been printed at 2 1
N has been printed at 2 2
N has been printed at 2 4
N has been printed at 2 5
N has been printed at 2 6
N has been printed at 2 7
N has been printed at 2 8
N has been printed at 2 9
N has been printed at 2 10
N has been printed at 3 0
N has been printed at 3 1
N has been printed at 3 2
N has been printed at 3 3
N has been printed at 3 5
N has been printed at 3 6
N has been printed at 3 7
N has been printed at 3 8
N has bee

,0,1,2,3,4,5,6,7,8,9,10
0,-14.593330,1.163395,1.686033,6.544806,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
1,1.686033,1.436073,-9.963087,6.840981,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
2,6.544806,-0.000000,6.840981,-14.617480,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000
3,-0.000000,-0.000000,-0.000000,-0.000000,-6.649543,-0.000000,-0.000000,1.994129,1.556487,3.098927,-0.000000
4,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,-5.368775,3.902050,-0.000000,-0.000000,-0.000000,1.466726
5,-0.000000,-0.000000,-0.000000,-0.000000,-0.000000,3.902050,-5.820552,1.918502,-0.000000,-0.000000,-0.000000
6,-0.000000,-0.000000,-0.000000,-0.000000,3.098927,-0.000000,-0.000000,-0.000000,2.538805,-6.808836,1.171104


In [125]:
L = LMatrix(G, B, V_mag, V_ang)

pd.DataFrame(L)

L has been printed at 0 1
L has been printed at 0 2
L has been printed at 0 3
L has been printed at 0 4
L has been printed at 0 5
L has been printed at 0 6
L has been printed at 1 0
L has been printed at 1 2
L has been printed at 1 3
L has been printed at 1 4
L has been printed at 1 5
L has been printed at 1 6
L has been printed at 2 0
L has been printed at 2 1
L has been printed at 2 3
L has been printed at 2 4
L has been printed at 2 5
L has been printed at 2 6
L has been printed at 3 0
L has been printed at 3 1
L has been printed at 3 2
L has been printed at 3 4
L has been printed at 3 5
L has been printed at 3 6
L has been printed at 4 0
L has been printed at 4 1
L has been printed at 4 2
L has been printed at 4 3
L has been printed at 4 5
L has been printed at 4 6
L has been printed at 5 0
L has been printed at 5 1
L has been printed at 5 2
L has been printed at 5 3
L has been printed at 5 4
L has been printed at 5 6
L has been printed at 6 0
L has been printed at 6 1
L has been p

,0,1,2,3,4,5,6
0,32.755953,-5.115838,-8.490435,0.000000,0.000000,0.000000,0.000000
1,-5.115838,28.755122,-21.578554,0.000000,-1.797979,0.000000,0.000000
2,-8.490435,-21.578554,38.350812,-3.967939,0.000000,0.000000,0.000000
3,0.000000,0.000000,-3.967939,17.120032,0.000000,0.000000,-6.102755
4,0.000000,-1.797979,0.000000,0.000000,15.028252,-10.365394,0.000000
5,0.000000,0.000000,0.000000,0.000000,-10.365394,14.613279,0.000000
6,0.000000,0.000000,0.000000,-6.102755,0.000000,0.000000,10.469205
